In [2]:
import yfinance as yf 
import pandas as pd 
import streamlit as st
from pandas.tseries.offsets import DateOffset

In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [6]:
tickers = tickers.to_list()

In [7]:
#use decorator on streamlit to cache data
@st.cache_data
def getdata():
    df = yf.download(tickers,start='2020-01-01')
    df = df['Close']
    return df 

2023-03-24 23:04:27.531 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [8]:
df = getdata()

2023-03-24 23:04:46.002 
  command:

    streamlit run C:\Users\chris\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-03-24 23:04:46.008 No runtime found, using MemoryCacheStorageManager


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [9]:
st.title('Index component performace for S&P 500')

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [10]:
n = st.number_input('Provide the performance horizon in months:',min_value=1,max_value=24)

In [30]:
def get_ret(df,n):
    previous_prices = df[:df.index[-1] - DateOffset(months=n)].tail(1).squeeze()
    recent_prices = df.loc[df.index[-1]]
    ret_df = (recent_prices/previous_prices) -1
    return previous_prices.name, ret_df 

In [32]:
date, ret_df = get_ret(df,n)

In [35]:
winners, losers = ret_df.nlargest(10), ret_df.nsmallest(10)

In [38]:
winners.name, losers.name = 'winners','losers'

In [37]:
st.table(winners)
st.table(losers)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [ ]:
winnerpick = st.selectbox('Pick a winner stock ticker to visualize performance:'winners.index)
st.line_chart(df[winnerpick][date:])
loserpick = st.selectbox('Pick a loser stock ticker to visualize performance:'losers.index)
st.line_chart(df[loserpick][date:])